In [1]:
import logging
import string
import os
import pickle
from pprint import pprint
import re 
import numpy as np
import pandas as pd
from IPython.display import clear_output
from more_itertools import chunked

# NLTK
import nltk
from nltk.corpus import stopwords

# Gensim
from gensim import corpora, models, similarities
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import gensim

# SpaCy 
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

# Logging if you want it
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# Suppressing it should you want to
logging.getLogger().setLevel(logging.CRITICAL)

# Suppressing warnings
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

#SKLearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction import text

In [3]:
path=os.listdir(r"C:\Users\Li\Desktop\中国（txt）")
datalist=[]

for i in path:
    domain= r"C:\\Users\\Li\\Desktop\\中国（txt）\\"+i
    #print(domain)
    with open(domain,"r",encoding="utf-8") as f:
        data=f.read()
        datalist.append(data)
len(datalist)

348

In [13]:
data2 = [re.sub(r'[^a-zA-Z\.:/]',' ',txt) for txt in datalist]
               
data1 = [txt.replace('\n', ' ') for txt in data2]

no_url = [re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', txt) for txt in data1]

no_line= [txt.replace('/', ' ').replace(':','').replace('ASIANREVIEW','').replace('.',' ') for txt in no_url]

data = [re.sub(r'\s+', ' ', txt) for txt in no_line]

data[5]

' S SOURCESPOTLIGHT By Huang Ge and Li Xuanmin China Nepal economic ties improve The year will be the early harvest year for China and Nepal to implement their leaders consen sus with economic and trade ties further developing through more cooperation on the Belt and Road Initiative BRI Chi nese Ambassador to Nepal Hou Yanqi told the Global Times Hou noted that relevant departments are now focusing on relevant work and have made a lot of progress especially in areas including a joint feasibility study of a bilateral free trade agreement choosing the location of a China Nepal cross border economic cooperation zone food import and export safety production capacity and investment Chinese President Xi Jinping paid a state visit to Nepal in October during which the two sides inked about agreements across a range of areas in cluding infrastructure health agriculture and tourism It also marks the first by a Chinese leader to the Himalayan country in years Hou said the visit has provided syste

In [14]:
def tokenizer(texts):
    for text in texts:
        yield(gensim.utils.simple_preprocess(text, deacc=True))

token_list=list(tokenizer(data))

In [15]:
# Build bigram and trigram models
# min_count = min amount of tokens/bigrams in corpus. Threshold = score-based; higher = fewer phrases.
bigram = gensim.models.Phrases(token_list, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[token_list], threshold=100)  

# `Phraser` must be built from an initial `Phrases` instance. 
# It then works faster while using much less memory. See https://radimrehurek.com/gensim/models/phrases.html
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

stop = set(stopwords.words('english')+['ASIANREVIEW','com','cn','globaltimes','VIEWPOINT','SOURCESPOTLIGHT'])


def remove_stopwords(texts):
    return [[word for word in doc if word not in stop] for doc in texts]
      
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for doc in texts:
        joined = nlp(" ".join(doc)) 
     
        texts_out.append([token.lemma_ for token in joined if token.pos_ in allowed_postags])
    return texts_out

In [16]:
tokens_no_stops = remove_stopwords(token_list)

print(tokens_no_stops[0][1])

seeks


In [17]:
# Form trigrams
trigrams = make_trigrams(tokens_no_stops)

# Do lemmatization 
lemmas = lemmatization(trigrams)

In [18]:
# Create Dictionary 
dictionary = corpora.Dictionary(lemmas)

# Create Corpus, i.e. Document-Term Matrix
corpus = [dictionary.doc2bow(text) for text in lemmas]

In [25]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=8, 
                                           random_state=100,
                                           # eval_every = 20, # this is evaluation, perplexity
                                           update_every=1,
                                           chunksize=500,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [26]:
lda_viz = gensimvis.prepare(lda_model, corpus, dictionary)
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.020415  0.021684       1        1  25.738787
2     -0.088179 -0.010134       2        1  20.543168
5     -0.006558 -0.046582       3        1  17.268430
6     -0.015054  0.117740       4        1   9.802547
1      0.013048 -0.086856       5        1   9.232814
3     -0.048978  0.064550       6        1   7.844402
0     -0.027896 -0.076739       7        1   5.377363
4      0.194034  0.016338       8        1   4.192488, topic_info=             Term        Freq       Total Category  logprob  loglift
729       myanmar  409.000000  409.000000  Default  30.0000  30.0000
2473      vaccine  116.000000  116.000000  Default  29.0000  29.0000
250       project  539.000000  539.000000  Default  28.0000  28.0000
2414         cpec  104.000000  104.000000  Default  27.0000  27.0000
49          india  294.000000  294.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
238          year   14.103404  532.200324   Topic8  -5.3259  -0.4587
380           day   11.148888  112.756756   Topic8  -5.5610   0.8580
28    development   13.161957  645.492642   Topic8  -5.3950  -0.7208
25    cooperation   12.704740  866.426641   Topic8  -5.4303  -1.0505
951        center   10.016448  131.341649   Topic8  -5.6681   0.5983

[596 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
2569      3  0.828269        accounting
5748      4  0.209343        adenovirus
5748      8  0.628029        adenovirus
5477      8  0.759693  adverse_reaction
4104      5  0.780818         aerospace
...     ...       ...               ...
549       2  0.020125              zone
549       3  0.503126              zone
549       4  0.241501              zone
549       5  0.120750              zone
549       6  0.020125              zone

[1463 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 3, 6, 7, 2, 4, 1, 5])